In [7]:
import pandas as pd

# Paso 1: Leer los archivos CSV
bacillus_df = pd.read_csv('common_genes_bacillus.csv')
fusobacterium_df = pd.read_csv('common_genes_fusobacterium.csv')
lactobacillus_df = pd.read_csv('common_genes_lactobacillus.csv')
propionibacterium_df = pd.read_csv('common_genes_propionibacterium.csv')

# Paso 2: Limpiar los nombres de los genes (convertir a mayúsculas y eliminar espacios)
def clean_gene_names(df):
    df['Gene'] = df['Gene'].str.strip().str.upper()
    return df

bacillus_df = clean_gene_names(bacillus_df)
fusobacterium_df = clean_gene_names(fusobacterium_df)
lactobacillus_df = clean_gene_names(lactobacillus_df)
propionibacterium_df = clean_gene_names(propionibacterium_df)

# Paso 3: Verificar si existen genes comunes entre cada par de DataFrames
print("Genes comunes entre Bacillus y Fusobacterium:", len(set(bacillus_df['Gene']).intersection(set(fusobacterium_df['Gene']))))
print("Genes comunes entre Bacillus y Lactobacillus:", len(set(bacillus_df['Gene']).intersection(set(lactobacillus_df['Gene']))))
print("Genes comunes entre Bacillus y Propionibacterium:", len(set(bacillus_df['Gene']).intersection(set(propionibacterium_df['Gene']))))

print("Genes comunes entre Fusobacterium y Lactobacillus:", len(set(fusobacterium_df['Gene']).intersection(set(lactobacillus_df['Gene']))))
print("Genes comunes entre Fusobacterium y Propionibacterium:", len(set(fusobacterium_df['Gene']).intersection(set(propionibacterium_df['Gene']))))

print("Genes comunes entre Lactobacillus y Propionibacterium:", len(set(lactobacillus_df['Gene']).intersection(set(propionibacterium_df['Gene']))))

# Paso 4: Hacer los merge de forma progresiva
# Primero probemos con Bacillus y Fusobacterium
common_genes_df = bacillus_df.merge(fusobacterium_df, on='Gene', suffixes=('_bacillus', '_fusobacterium'), how='inner')

# Comprobar cuántos genes comunes se encontraron
print("Genes comunes después de Bacillus y Fusobacterium:", len(common_genes_df))

# Luego Lactobacillus
common_genes_df = common_genes_df.merge(lactobacillus_df, on='Gene', suffixes=('', '_lactobacillus'), how='inner')
print("Genes comunes después de agregar Lactobacillus:", len(common_genes_df))

# Finalmente Propionibacterium
common_genes_df = common_genes_df.merge(propionibacterium_df, on='Gene', suffixes=('', '_propionibacterium'), how='inner')
print("Genes comunes después de agregar Propionibacterium:", len(common_genes_df))

# Paso 5: Ver las columnas que tenemos
print("Columnas después del último merge:", common_genes_df.columns)

# Paso 6: Exportar a CSV solo si hay datos
if not common_genes_df.empty:
    # Combinar las categorías
    category_columns = [col for col in common_genes_df.columns if 'Category' in col]
    common_genes_df['Category'] = common_genes_df[category_columns].apply(lambda x: ', '.join(x.dropna().unique()), axis=1)

    # Exportar el DataFrame final a CSV y Excel
    common_genes_df = common_genes_df[['Gene', 'Category']]
    common_genes_df.to_csv('common_genes_final.csv', index=False)
    common_genes_df.to_excel('common_genes_final.xlsx', index=False)
    print("Archivo exportado correctamente.")
else:
    print("No se encontraron genes comunes.")


Genes comunes entre Bacillus y Fusobacterium: 2
Genes comunes entre Bacillus y Lactobacillus: 1
Genes comunes entre Bacillus y Propionibacterium: 0
Genes comunes entre Fusobacterium y Lactobacillus: 0
Genes comunes entre Fusobacterium y Propionibacterium: 0
Genes comunes entre Lactobacillus y Propionibacterium: 0
Genes comunes después de Bacillus y Fusobacterium: 2
Genes comunes después de agregar Lactobacillus: 0
Genes comunes después de agregar Propionibacterium: 0
Columnas después del último merge: Index(['Gene', 'Category_bacillus', 'Category_fusobacterium', 'Category',
       'Category_propionibacterium'],
      dtype='object')
No se encontraron genes comunes.


In [20]:
import pandas as pd

# Paso 1: Leer los archivos CSV
bacillus_df = pd.read_csv('common_genes_bacillus.csv')
fusobacterium_df = pd.read_csv('common_genes_fusobacterium.csv')
lactobacillus_df = pd.read_csv('common_genes_lactobacillus.csv')
propionibacterium_df = pd.read_csv('common_genes_propionibacterium.csv')

# Paso 2: Limpiar los nombres de los genes (convertir a mayúsculas y eliminar espacios)
def clean_gene_names(df):
    df['Gene'] = df['Gene'].str.strip().str.upper()
    return df

bacillus_df = clean_gene_names(bacillus_df)
fusobacterium_df = clean_gene_names(fusobacterium_df)
lactobacillus_df = clean_gene_names(lactobacillus_df)
propionibacterium_df = clean_gene_names(propionibacterium_df)

# Crear una lista para almacenar los datos de genes comunes
common_genes_list = []

# Paso 3: Identificar genes comunes entre Bacillus y Fusobacterium
bacillus_genes = set(bacillus_df['Gene'])
fusobacterium_genes = set(fusobacterium_df['Gene'])
common_genes = bacillus_genes.intersection(fusobacterium_genes)
print(f"Genes comunes entre Bacillus y Fusobacterium: {len(common_genes)}")

# Filtrar los genes comunes y agregar a la lista
if common_genes:
    bacillus_common = bacillus_df[bacillus_df['Gene'].isin(common_genes)]
    fusobacterium_common = fusobacterium_df[fusobacterium_df['Gene'].isin(common_genes)]
    
    for gene in common_genes:
        bacillus_row = bacillus_common[bacillus_common['Gene'] == gene]
        fusobacterium_row = fusobacterium_common[fusobacterium_common['Gene'] == gene]
        
        # Combinando categorías y eliminando duplicados
        categories = set([
            bacillus_row['Category'].values[0] if not bacillus_row.empty else '',
            fusobacterium_row['Category'].values[0] if not fusobacterium_row.empty else ''
        ])
        category = ', '.join(filter(None, categories))
        
        common_genes_list.append({
            'Gene': gene,
            'Category': category,
            'Present_In': 'Bacillus, Fusobacterium'
        })

# Paso 4: Agregar información de Lactobacillus
lactobacillus_genes = set(lactobacillus_df['Gene'])
common_genes = [item['Gene'] for item in common_genes_list]
new_common_genes = set(common_genes).intersection(lactobacillus_genes)
print(f"Genes comunes después de agregar Lactobacillus: {len(new_common_genes)}")

if new_common_genes:
    lactobacillus_common = lactobacillus_df[lactobacillus_df['Gene'].isin(new_common_genes)]
    
    # Actualizar la lista con información de Lactobacillus
    for gene in new_common_genes:
        for item in common_genes_list:
            if item['Gene'] == gene:
                lactobacillus_row = lactobacillus_common[lactobacillus_common['Gene'] == gene]
                
                # Actualizar categorías y presencia
                existing_categories = set(item['Category'].split(', '))
                new_categories = set([
                    lactobacillus_row['Category'].values[0] if not lactobacillus_row.empty else ''
                ])
                new_category = ', '.join(sorted(existing_categories.union(new_categories)))
                
                new_present_in = item['Present_In'] + ', Lactobacillus'
                item['Category'] = new_category
                item['Present_In'] = new_present_in

# Paso 5: Agregar información de Propionibacterium
propionibacterium_genes = set(propionibacterium_df['Gene'])
common_genes = [item['Gene'] for item in common_genes_list]
new_common_genes = set(common_genes).intersection(propionibacterium_genes)
print(f"Genes comunes después de agregar Propionibacterium: {len(new_common_genes)}")

if new_common_genes:
    propionibacterium_common = propionibacterium_df[propionibacterium_df['Gene'].isin(new_common_genes)]
    
    # Actualizar la lista con información de Propionibacterium
    for gene in new_common_genes:
        for item in common_genes_list:
            if item['Gene'] == gene:
                propionibacterium_row = propionibacterium_common[propionibacterium_common['Gene'] == gene]
                
                # Actualizar categorías y presencia
                existing_categories = set(item['Category'].split(', '))
                new_categories = set([
                    propionibacterium_row['Category'].values[0] if not propionibacterium_row.empty else ''
                ])
                new_category = ', '.join(sorted(existing_categories.union(new_categories)))
                
                new_present_in = item['Present_In'] + ', Propionibacterium'
                item['Category'] = new_category
                item['Present_In'] = new_present_in

# Convertir la lista a DataFrame
common_genes_df = pd.DataFrame(common_genes_list)

# Eliminar duplicados en la columna 'Category' (por cada 'Gene')
common_genes_df['Category'] = common_genes_df.groupby('Gene')['Category'].transform(lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))))

# Eliminar duplicados en base a 'Gene' (mantener la primera aparición)
common_genes_df = common_genes_df.drop_duplicates(subset='Gene')

# Paso 6: Exportar el DataFrame final a CSV y Excel
common_genes_df.to_csv('common_genes_final.csv', index=False)
common_genes_df.to_excel('common_genes_final.xlsx', index=False)

print("Archivo exportado correctamente con genes presentes en al menos dos DataFrames y su presencia indicada.")


Genes comunes entre Bacillus y Fusobacterium: 2
Genes comunes después de agregar Lactobacillus: 0
Genes comunes después de agregar Propionibacterium: 0
Archivo exportado correctamente con genes presentes en al menos dos DataFrames y su presencia indicada.


In [19]:
common_genes_df

,Gene,Category,Present_In
0,PLK2,"Cancer, Rare Diseases, Associated with Dysbios...","Bacillus, Fusobacterium"
1,THBS1,"Cancer, Rare Diseases, Associated with Dysbios...","Bacillus, Fusobacterium"


Genes without filter / Same Results

In [22]:
# Paso 1: Leer los archivos CSV
bacillus_df = pd.read_csv('genes_multiple_categories_bacillus.csv')
fusobacterium_df = pd.read_csv('genes_multiple_categories_fusobacterium.csv')
lactobacillus_df = pd.read_csv('genes_multiple_categories_lactobacillus.csv')
propionibacterium_df = pd.read_csv('genes_multiple_categories_propionibacterium.csv')

# Paso 2: Limpiar los nombres de los genes (convertir a mayúsculas y eliminar espacios)
def clean_gene_names(df):
    df['Gene'] = df['Gene'].str.strip().str.upper()
    return df

bacillus_df = clean_gene_names(bacillus_df)
fusobacterium_df = clean_gene_names(fusobacterium_df)
lactobacillus_df = clean_gene_names(lactobacillus_df)
propionibacterium_df = clean_gene_names(propionibacterium_df)

# Crear una lista para almacenar los datos de genes comunes
common_genes_list = []

# Paso 3: Identificar genes comunes entre Bacillus y Fusobacterium
bacillus_genes = set(bacillus_df['Gene'])
fusobacterium_genes = set(fusobacterium_df['Gene'])
common_genes = bacillus_genes.intersection(fusobacterium_genes)
print(f"Genes comunes entre Bacillus y Fusobacterium: {len(common_genes)}")

# Filtrar los genes comunes y agregar a la lista
if common_genes:
    bacillus_common = bacillus_df[bacillus_df['Gene'].isin(common_genes)]
    fusobacterium_common = fusobacterium_df[fusobacterium_df['Gene'].isin(common_genes)]
    
    for gene in common_genes:
        bacillus_row = bacillus_common[bacillus_common['Gene'] == gene]
        fusobacterium_row = fusobacterium_common[fusobacterium_common['Gene'] == gene]
        
        # Combinando categorías y eliminando duplicados
        categories = set([
            bacillus_row['Category'].values[0] if not bacillus_row.empty else '',
            fusobacterium_row['Category'].values[0] if not fusobacterium_row.empty else ''
        ])
        category = ', '.join(filter(None, categories))
        
        common_genes_list.append({
            'Gene': gene,
            'Category': category,
            'Present_In': 'Bacillus, Fusobacterium'
        })

# Paso 4: Agregar información de Lactobacillus
lactobacillus_genes = set(lactobacillus_df['Gene'])
common_genes = [item['Gene'] for item in common_genes_list]
new_common_genes = set(common_genes).intersection(lactobacillus_genes)
print(f"Genes comunes después de agregar Lactobacillus: {len(new_common_genes)}")

if new_common_genes:
    lactobacillus_common = lactobacillus_df[lactobacillus_df['Gene'].isin(new_common_genes)]
    
    # Actualizar la lista con información de Lactobacillus
    for gene in new_common_genes:
        for item in common_genes_list:
            if item['Gene'] == gene:
                lactobacillus_row = lactobacillus_common[lactobacillus_common['Gene'] == gene]
                
                # Actualizar categorías y presencia
                existing_categories = set(item['Category'].split(', '))
                new_categories = set([
                    lactobacillus_row['Category'].values[0] if not lactobacillus_row.empty else ''
                ])
                new_category = ', '.join(sorted(existing_categories.union(new_categories)))
                
                new_present_in = item['Present_In'] + ', Lactobacillus'
                item['Category'] = new_category
                item['Present_In'] = new_present_in

# Paso 5: Agregar información de Propionibacterium
propionibacterium_genes = set(propionibacterium_df['Gene'])
common_genes = [item['Gene'] for item in common_genes_list]
new_common_genes = set(common_genes).intersection(propionibacterium_genes)
print(f"Genes comunes después de agregar Propionibacterium: {len(new_common_genes)}")

if new_common_genes:
    propionibacterium_common = propionibacterium_df[propionibacterium_df['Gene'].isin(new_common_genes)]
    
    # Actualizar la lista con información de Propionibacterium
    for gene in new_common_genes:
        for item in common_genes_list:
            if item['Gene'] == gene:
                propionibacterium_row = propionibacterium_common[propionibacterium_common['Gene'] == gene]
                
                # Actualizar categorías y presencia
                existing_categories = set(item['Category'].split(', '))
                new_categories = set([
                    propionibacterium_row['Category'].values[0] if not propionibacterium_row.empty else ''
                ])
                new_category = ', '.join(sorted(existing_categories.union(new_categories)))
                
                new_present_in = item['Present_In'] + ', Propionibacterium'
                item['Category'] = new_category
                item['Present_In'] = new_present_in

# Convertir la lista a DataFrame
common_genes_df = pd.DataFrame(common_genes_list)

# Eliminar duplicados en la columna 'Category' (por cada 'Gene')
common_genes_df['Category'] = common_genes_df.groupby('Gene')['Category'].transform(lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))))

# Eliminar duplicados en base a 'Gene' (mantener la primera aparición)
common_genes_df = common_genes_df.drop_duplicates(subset='Gene')

# Paso 6: Exportar el DataFrame final a CSV y Excel
common_genes_df.to_csv('genes_multiple_categories_final.csv', index=False)
common_genes_df.to_excel('genes_multiple_categories_final.xlsx', index=False)

print("Archivo exportado correctamente con genes presentes en al menos dos DataFrames y su presencia indicada.")

Genes comunes entre Bacillus y Fusobacterium: 2
Genes comunes después de agregar Lactobacillus: 0
Genes comunes después de agregar Propionibacterium: 0
Archivo exportado correctamente con genes presentes en al menos dos DataFrames y su presencia indicada.
